In [1]:
import os
from pathlib import Path
import pandas as pd
import json
from epjson.epjson_idf import EpJsonIDF
from epjson.simulate_cluster import SimulateCluster
import datetime
from IPython.display import clear_output

def run_single_building(
        config_dict, 
        output_dir, 
        weight_map,
        resource_dir=Path("tests/resources"), 
        idf_dir=Path("tests/resources/idfs/"),
        eplus_loc=Path("C:\EnergyPlusV22-2-0"), 
        update_epsjon=True
        ):
    
    weights_df = pd.read_json(resource_dir / "shoebox-weights.json")
    weights_df = weights_df[weights_df["building_id"] == config_dict["building_id"]]
    idfs = set(weights_df.ShoeboxPath.to_list())

    ### MUTATIONS ###

    # Get retrofit scenario template data
    scenario_dir = resource_dir / f"templates_RETRO_{config_dict['retrofit_scenario']}_SCHED_{config_dict['schedules_scenario']}_LAB_{config_dict['lab_scenario']}.csv"
    features_df = pd.read_csv(scenario_dir)
    print(scenario_dir)
    with open(resource_dir / "template_features_map.json", "r") as f:
        features_map = json.load(f)
    with open(resource_dir / "schedule_bunches.json", "r") as f:
        schedules = json.load(f)
        
    if update_epsjon:
        for i, idf in enumerate(idfs):
            try:
                idf_path = idf_dir / idf
                epjson = EpJsonIDF.from_epjson(idf_path.with_suffix(".epjson"), eplus_loc=eplus_loc)
                features_series = features_df[(features_df.TYPOLOGY == config_dict["TYPOLOGY"]) & (features_df.AGE_ERA == config_dict["AGE_ERA"])].reset_index(drop=True)
                features_series = features_series.iloc[0]
                # Set DHW flow rate based on sb area
                peak_dhw_pp = features_series["DHWFlowRate"]
                occupants = features_series["PeopleDensity"] * (weights_df.iloc[i]["core_area"]+weights_df.iloc[i]["perim_area"])
                features_series["DHWFlowRate"] = peak_dhw_pp * occupants

                cols = set(features_series.index) & set(features_map.keys())

                epjson.replace_features(features_series[cols], features_map)
                epjson.update_schedules(schedules)
                epjson.add([(
                    "Output:SQLite",
                    {"Output:SQLite 1": {"option_type": "Simple"}}
                    )])
                # set core infiltration to 1/4 of perimeter (only exterior exposure is through roof)
                infil_baseline = features_series["Infiltration"]
                infil_fact = 0.25
                epjson.epjson["ZoneInfiltration:DesignFlowRate"]["CoreInfiltration"].update(
                    {"air_changes_per_hour": infil_baseline*infil_fact}
                )
                epjson.epjson["WaterUse:Equipment"]["DHW Perimeter"].update(
                    {"peak_flow_rate": 0}
                )
                
                # new_path = replace_shoebox_path(idf_path).with_suffix(".epJSON")
                new_path = output_dir / Path(idf).with_suffix(".epJSON")
                epjson.save_as(new_path)
                epjson.save_idf(output_path=new_path.parent) #suffix="_test",

            except Exception as e:
                print(f"FAILED ON {idf}")
                raise e
    clear_output()
    ### SIMULATION ###
    # Get new paths for updated shoeboxes
    # new_idfs = weights_df["ShoeboxPath"].apply(replace_shoebox_path)
    new_idfs = weights_df["ShoeboxPath"].apply(lambda x: output_dir / Path(x))
    weights_df["ShoeboxPath"] = new_idfs
    
    print("BUILDING CLUSTER")
    idf_cluster = SimulateCluster(
        idf_list = set(new_idfs),
        epw = config_dict["epw"],
        weights_df = weights_df,
        building_col = "building_id",
        eplus_location=eplus_loc,
        weight_map = weight_map
    )
    print("SIMULATING CLUSTER")
    errors = idf_cluster.parallel_simulate() # Fix errors dict

    print("PROCESSING CLUSTER RESULTS")
    ### POST-PROCESS ###
    r = idf_cluster.fetch_building_results_parallel([config_dict["building_id"]])
    clear_output()
    
    return r, errors, idf_cluster

c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package energy-pandas is out of date. Your version is 0.3.3, the latest is 0.3.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
weights_df = pd.read_json(Path("tests/resources") / "shoebox-weights.json")

In [3]:
def run_building_with_lab(bid, retrofit="baseline", schedules="baseline", labs="baseline"):
    b_series = weights_df[weights_df.building_id == bid].iloc[0]
    config_dict = {
        "building_id": bid,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\BaselineFile.epw",
        "retrofit_scenario": retrofit,
        "schedules_scenario": schedules,
        "lab_scenario": labs,
        "TYPOLOGY": b_series["TYPOLOGY"],
        "AGE_ERA": b_series["AGE_ERA"],
        "lab_weight": b_series["LabsAreaWeight"]
    }
    print(f'Running building {bid} ({b_series["TYPOLOGY"]}, {b_series["AGE_ERA"]}, {round(b_series["LabsAreaWeight"]*100)} lab)')

    sb_weight_map = {
        "Perimeter": "PerimeterAreaWeight",
        "Core": "CoreAreaWeight"
    }

    res, err, cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    res = res[config_dict["building_id"]]
    res.set_index(["DateTime", "BuildingId"], inplace=True)

    if config_dict["lab_weight"] > 0:
        lab_config = config_dict.copy()
        lab_config["TYPOLOGY"] = "LAB_ONLY"
        lab_res, lab_err, lab_cluster = run_single_building(lab_config, 
                                    weight_map = sb_weight_map,
                                    output_dir = Path(os.getcwd()) / "eplus", 
                                    idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                    update_epsjon=True
                                    )
        lab_res = lab_res[config_dict["building_id"]]
        lab_res.set_index(["DateTime", "BuildingId"], inplace=True)
        res = (res * (1-config_dict["lab_weight"])) + (lab_res * config_dict["lab_weight"])
    res.reset_index(inplace=True)
    return res

In [4]:
# small_lab = run_building_with_lab("62")

In [5]:
# small_lab.sum()

Run all archetypes

In [6]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY"]).first().reset_index()
# sbs = sbs.groupby(["TYPOLOGY"]).sample(n=2).reset_index()
sbs = sbs[sbs.TYPOLOGY.isin(["Athletic", "Residential"])]
# sbs = sbs[sbs.building_id.isin(["1", "14", "W41", "50", "N10"])]
# sbs = sbs[sbs.TYPOLOGY == "Lab and Mixed Use"]
# sbs = sbs.groupby(["building_id"]).first().reset_index()
sbs.shape


(2, 45)

In [7]:
# hourly_baseline = pd.DataFrame(columns=list(res_hourly.values())[0].columns)
# for idx, df in res_hourly.items():
#     df.set_index(["TYPOLOGY", "AGE_ERA", "BuildingId", "DateTime"])
#     hourly_baseline = pd.concat([hourly_baseline, df], axis=0)

In [8]:
# hourly_baseline.to_csv("baseline_results_samples.csv")

In [9]:
res = {}
res_hourly = {}
for i, row in sbs.iterrows():
    bldg_res = run_building_with_lab(row.building_id, retrofit="baseline", schedules="baseline", labs="baseline")
    print(row.TYPOLOGY, row.AGE_ERA)
    res[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = bldg_res.sum()
    bldg_res["DateTime"] = pd.date_range(start=datetime.datetime(year=2025, day=1, month=1, hour=0), periods=8760, freq="H")
    bldg_res["TYPOLOGY"] = row.TYPOLOGY
    bldg_res["AGE_ERA"] = row.AGE_ERA
    res_hourly[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = bldg_res

clear_output()

# BASELINE
norm_cols = ['Zone Lights Electricity Energy_norm',
       'Electric Equipment Electricity Energy_norm',
       'Zone Ideal Loads Supply Air Total Heating Energy_norm',
       'Zone Ideal Loads Supply Air Total Cooling Energy_norm',
       'Water Use Equipment Heating Energy_norm']
df = pd.DataFrame.from_dict(res).T[norm_cols]
df["Cooling"] = df['Zone Ideal Loads Supply Air Total Cooling Energy_norm']
df["Elec"] = df[['Zone Lights Electricity Energy_norm', 'Electric Equipment Electricity Energy_norm']].sum(axis=1)
df["Heating"] = df[['Zone Ideal Loads Supply Air Total Heating Energy_norm', 'Water Use Equipment Heating Energy_norm']].sum(axis=1)
df #[['Cooling', 'Elec', 'Heating']]

,,,Zone Lights Electricity Energy_norm,Electric Equipment Electricity Energy_norm,Zone Ideal Loads Supply Air Total Heating Energy_norm,Zone Ideal Loads Supply Air Total Cooling Energy_norm,Water Use Equipment Heating Energy_norm,Cooling,Elec,Heating
Athletic,1945,51,109.659925,53.002297,119.258733,64.280576,23.27645,64.280576,162.662221,142.535183
Residential,1945,62,19.137394,101.667853,74.705873,74.756342,32.904126,74.756342,120.805248,107.609998


In [10]:
res = {}
res_hourly = {}
for i, row in sbs.iterrows():
    bldg_res = run_building_with_lab(row.building_id, retrofit="partial", schedules="baseline", labs="baseline")
    print(row.TYPOLOGY, row.AGE_ERA)
    res[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = bldg_res.sum()
    bldg_res["DateTime"] = pd.date_range(start=datetime.datetime(year=2025, day=1, month=1, hour=0), periods=8760, freq="H")
    bldg_res["TYPOLOGY"] = row.TYPOLOGY
    bldg_res["AGE_ERA"] = row.AGE_ERA
    res_hourly[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = bldg_res

clear_output()

# PARTIAL
norm_cols = ['Zone Lights Electricity Energy_norm',
       'Electric Equipment Electricity Energy_norm',
       'Zone Ideal Loads Supply Air Total Heating Energy_norm',
       'Zone Ideal Loads Supply Air Total Cooling Energy_norm',
       'Water Use Equipment Heating Energy_norm']
df = pd.DataFrame.from_dict(res).T[norm_cols]
df["Cooling"] = df['Zone Ideal Loads Supply Air Total Cooling Energy_norm']
df["Elec"] = df[['Zone Lights Electricity Energy_norm', 'Electric Equipment Electricity Energy_norm']].sum(axis=1)
df["Heating"] = df[['Zone Ideal Loads Supply Air Total Heating Energy_norm', 'Water Use Equipment Heating Energy_norm']].sum(axis=1)
df #[['Cooling', 'Elec', 'Heating']]

,,,Zone Lights Electricity Energy_norm,Electric Equipment Electricity Energy_norm,Zone Ideal Loads Supply Air Total Heating Energy_norm,Zone Ideal Loads Supply Air Total Cooling Energy_norm,Water Use Equipment Heating Energy_norm,Cooling,Elec,Heating
Athletic,1945,51,54.829962,53.002297,89.495369,47.574316,18.62815,47.574316,107.832259,108.123519
Residential,1945,62,7.425309,30.104981,110.098304,35.291101,9.184163,35.291101,37.530290,119.282467


In [11]:
res = {}
res_hourly = {}
for i, row in sbs.iterrows():
    bldg_res = run_building_with_lab(row.building_id, retrofit="full", schedules="baseline", labs="baseline")
    print(row.TYPOLOGY, row.AGE_ERA)
    res[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = bldg_res.sum()
    bldg_res["DateTime"] = pd.date_range(start=datetime.datetime(year=2025, day=1, month=1, hour=0), periods=8760, freq="H")
    bldg_res["TYPOLOGY"] = row.TYPOLOGY
    bldg_res["AGE_ERA"] = row.AGE_ERA
    res_hourly[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = bldg_res

# FULL
norm_cols = ['Zone Lights Electricity Energy_norm',
       'Electric Equipment Electricity Energy_norm',
       'Zone Ideal Loads Supply Air Total Heating Energy_norm',
       'Zone Ideal Loads Supply Air Total Cooling Energy_norm',
       'Water Use Equipment Heating Energy_norm']
df = pd.DataFrame.from_dict(res).T[norm_cols]
df["Cooling"] = df['Zone Ideal Loads Supply Air Total Cooling Energy_norm']
df["Elec"] = df[['Zone Lights Electricity Energy_norm', 'Electric Equipment Electricity Energy_norm']].sum(axis=1)
df["Heating"] = df[['Zone Ideal Loads Supply Air Total Heating Energy_norm', 'Water Use Equipment Heating Energy_norm']].sum(axis=1)
df #[['Cooling', 'Elec', 'Heating']]

Residential 1945


C:\Users\zoelh\AppData\Local\Temp\ipykernel_92672\354733910.py:6: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  res[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = bldg_res.sum()


,,,Zone Lights Electricity Energy_norm,Electric Equipment Electricity Energy_norm,Zone Ideal Loads Supply Air Total Heating Energy_norm,Zone Ideal Loads Supply Air Total Cooling Energy_norm,Water Use Equipment Heating Energy_norm,Cooling,Elec,Heating
Athletic,1945,51,54.829962,53.002297,7.886454,34.556872,18.62815,34.556872,107.832259,26.514604
Residential,1945,62,7.425309,30.104981,7.801229,15.572443,9.184163,15.572443,37.530290,16.985393


# Compare other scenarios

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
partialres = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "partial",
        "schedules_scenario": "baseline",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    partialres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()
    

In [ ]:
# pd.DataFrame.from_dict(partialres).T[norm_cols]

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
fullres = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "full",
        "schedules_scenario": "baseline",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    fullres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()
    

In [62]:
pd.DataFrame.from_dict(fullres).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                             87.354719   
                     1980 W31                             78.61925   
Lab and Mixed Use    1945 1                              52.412832   
                     1980 13                             52.412831   
                     2015 12                              52.41283   
Office and Mixed Use 1945 50                              52.41283   
                     1980 W11                             25.33287   
Residential          1945 62                              7.555455   
                     1980 NW30                            7.555456   
                     2015 E37                             7.555455   
Support Dominant     1945 42                             13.285197   
                     1980 E23                            13.285197   
                     2015 42C                            13.285196   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   436.773593   
                     1980 W31                                  436.773613   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   349.418875   
                     2015 12                                   105.553617   
Office and Mixed Use 1945 50                                   480.450939   
                     1980 W11                                  349.418901   
Residential          1945 62                                    30.920002   
                     1980 NW30                                  30.920005   
                     2015 E37                                   30.920003   
Support Dominant     1945 42                                   132.851972   
                     1980 E23                                  132.851969   
                     2015 42C                                  132.851964   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                                  0.0      
                     1980 W31                                                 0.0      
Lab and Mixed Use    1945 1                                                   0.0      
                     1980 13                                             0.002462      
                     2015 12                                             1.587394      
Office and Mixed Use 1945 50                                                  0.0      
                     1980 W11                                            0.040111      
Residential          1945 62                                            30.460753      
                     1980 NW30                                           12.12782      
                     2015 E37                                           11.881545      
Support Dominant     1945 42                                             6.900067      
                     1980 E23                                            6.243716      
                     2015 42C                                             4.43932      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                            61.736831      
                     1980 W31                                           59.307497      
Lab and Mixed Use    1945 1                                              88.42994      
                     1980 13                                            61.598259      
                     2015 12                                            41.221423      
Office and Mixed Use 1945 50                                            58.401299      
                     1980 W11                                           66.612453      
Residential          1945 62      

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
allfullres = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "full",
        "schedules_scenario": "full",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    allfullres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()


In [64]:
pd.DataFrame.from_dict(allfullres).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                             87.354719   
                     1980 W31                             78.61925   
Lab and Mixed Use    1945 1                              32.648827   
                     1980 13                             32.648826   
                     2015 12                             32.648826   
Office and Mixed Use 1945 50                             32.648825   
                     1980 W11                            32.648829   
Residential          1945 62                              7.555455   
                     1980 NW30                            7.555456   
                     2015 E37                             7.555455   
Support Dominant     1945 42                             27.207356   
                     1980 E23                            27.207355   
                     2015 42C                            27.207354   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   436.773593   
                     1980 W31                                  436.773613   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   349.418875   
                     2015 12                                   105.553617   
Office and Mixed Use 1945 50                                   480.450939   
                     1980 W11                                  349.418901   
Residential          1945 62                                    30.920002   
                     1980 NW30                                  30.920005   
                     2015 E37                                   30.920003   
Support Dominant     1945 42                                   132.851972   
                     1980 E23                                  132.851969   
                     2015 42C                                  132.851964   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                                  0.0      
                     1980 W31                                                 0.0      
Lab and Mixed Use    1945 1                                                   0.0      
                     1980 13                                                  0.0      
                     2015 12                                             0.100986      
Office and Mixed Use 1945 50                                                  0.0      
                     1980 W11                                                 0.0      
Residential          1945 62                                            17.905335      
                     1980 NW30                                           5.145644      
                     2015 E37                                            4.954598      
Support Dominant     1945 42                                             0.386677      
                     1980 E23                                            0.336192      
                     2015 42C                                            0.211382      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                            61.677695      
                     1980 W31                                           59.276548      
Lab and Mixed Use    1945 1                                             71.013608      
                     1980 13                                            47.978383      
                     2015 12                                            33.496247      
Office and Mixed Use 1945 50                                            44.876122      
                     1980 W11                                           55.365314      
Residential          1945 62      